In [6]:
import numpy as np
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F

from skorch import NeuralNetClassifier
from sklearn.model_selection import cross_val_score


X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)

class MyModule(nn.Module):
    def __init__(self, num_units=10, nonlin=F.relu):
        super(MyModule, self).__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X


net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

net.fit(X, y)
y_proba = net.predict_proba(X)

#In an sklearn Pipeline:

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


pipe = Pipeline([
    ('scale', StandardScaler()),
    ('net', net),
])

pipe.fit(X, y)
y_proba = pipe.predict_proba(X)
print(y_proba)

#With grid search

# from sklearn.model_selection import GridSearchCV

# params = {
#     'lr': [0.01, 0.02],
#     'max_epochs': [10, 20],
#     'module__num_units': [10, 20],
# }
# gs = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy')

# gs.fit(X, y)
# print(gs.best_score_, gs.best_params_)

# cv_score = cross_val_score(pipe, X, y, cv = 5)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6843       0.5900        0.6725  0.0157
      2        0.6616       0.6350        0.6602  0.0136
      3        0.6466       0.6500        0.6550  0.0142
      4        0.6486       0.6650        0.6446  0.0133
      5        0.6346       0.6750        0.6362  0.0126
      6        0.6266       0.6700        0.6254  0.0134
      7        0.5980       0.6950        0.6136  0.0119
      8        0.6079       0.6900        0.6078  0.0121
      9        0.5969       0.7050        0.5960  0.0123
     10        0.5932       0.7050        0.5936  0.0117
Re-initializing module!
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6901       0.5350        0.6886  0.0111
      2        0.6870       0.5450        0.6873  0.0120
      3        0.6864       0.5900        0.6865  0.0121
      4